In [1]:
from main import *

GetDist Version: 1.5.1, Matplotlib version: 3.5.1


I am following the Cobaya's example for a cosmological likelihood

First of all, we will need to simulate the fictitious power spectrum of the fictitious sky that we would measure with our fictitious experiment, once we have accounted for noise and beam corrections. To do that, we choose a set of true cosmological parameters in the sky, and then use a model 
to compute the corresponding power spectrum, up to some reasonable $ \ell_{max}$

In [24]:
%matplotlib inline

In [4]:
fiducial_params = {
    'ombh2': 0.022, 'omch2': 0.12, 'H0': 68, 'tau': 0.07,
    'As': 2.2e-9, 'ns': 0.96,
    'mnu': 0.06, 'nnu': 3.046}
l_max = 1000
packages_path = '/path/to/your/packages'
info_fiducial = {
    'params': fiducial_params,
    'likelihood': {'one': None},
    'theory': {'camb': {"extra_args": {"num_massive_neutrinos": 1}}},
    'packages_path': packages_path}


In [5]:
from cobaya.model import get_model
model_fiducial = get_model(info_fiducial)

[camb] `camb` module loaded successfully from /home/akozameh/miniconda3/envs/CMB/lib/python3.10/site-packages/camb
[model] *WARNING* Theories {camb, camb.transfers} do not appear to be actually used for anything


In [6]:
# Declare our desired theory product
# (there is no cosmological likelihood doing it for us)
model_fiducial.add_requirements({"Cl": {'tt': l_max}})

In [7]:
# Compute and extract the CMB power spectrum
# (In muK^-2, without l(l+1)/(2pi) factor)
# notice the empty dictionary below: all parameters are fixed
model_fiducial.logposterior({})
Cls = model_fiducial.provider.get_Cl(ell_factor=False, units="muK2")

In [8]:
# Our fiducial power spectrum
Cl_est = Cls['tt'][:l_max + 1]

In [26]:
import numpy as np
import matplotlib.pyplot as plt

_do_plot = True  # Set to True to allow plotting
_plot_name = "plot.png"  # Ensure you have a name for the output plot

def my_like(
        # Parameters that we may sample over (or not)
        noise_std_pixel=20,  # muK
        beam_FWHM=0.25,  # deg
        # Keyword through which the cobaya likelihood instance will be passed.
        _self=None):
    # Noise spectrum, beam-corrected
    healpix_Nside = 512
    pixel_area_rad = np.pi / (3 * healpix_Nside ** 2)
    weight_per_solid_angle = (noise_std_pixel ** 2 * pixel_area_rad) ** -1
    beam_sigma_rad = beam_FWHM / np.sqrt(8 * np.log(2)) * np.pi / 180.
    l_max = 1000  # Example value for l_max, make sure this is defined
    ells = np.arange(l_max + 1)
    Nl = np.exp((ells * beam_sigma_rad) ** 2) / weight_per_solid_angle
    # Cl of the map: data + noise
    Cl_est = np.ones_like(ells)  # Placeholder for your actual Cl_est values
    Cl_map = Cl_est + Nl
    # Request the Cl from the provider
    Cl_theo = _self.provider.get_Cl(ell_factor=False, units="muK2")['tt'][:l_max + 1]
    Cl_map_theo = Cl_theo + Nl
    # Auxiliary plot
    if _do_plot:
        ell_factor = ells * (ells + 1) / (2 * np.pi)
        plt.figure()
        plt.plot(ells[2:], (Cl_theo * ell_factor)[2:], label=r'Theory $C_\ell$')
        plt.plot(ells[2:], (Cl_est * ell_factor)[2:], label=r'Estimated $C_\ell$', ls="--")
        plt.plot(ells[2:], (Cl_map * ell_factor)[2:], label=r'Map $C_\ell$')
        plt.plot(ells[2:], (Nl * ell_factor)[2:], label='Noise')
        plt.legend()
        plt.ylim([0, 1000])
        plt.savefig('plot_aug.png')
        plt.show()
    # ----------------
    # Compute the log-likelihood
    V = Cl_map[2:] / Cl_map_theo[2:]
    logp = np.sum((2 * ells[2:] + 1) * (-V / 2 + 1 / 2. * np.log(V)))
    # Set our derived parameter
    derived = {'Map_Cl_at_500': Cl_map[500]}
    return logp, derived


In [31]:
info = {
    'params': {
        # Fixed
        'ombh2': 0.022, 'omch2': 0.12, 'H0': 68, 'tau': 0.07,
        'mnu': 0.06, 'nnu': 3.046,
        # Sampled
        'As': {'prior': {'min': 1e-9, 'max': 4e-9}, 'latex': 'A_s'},
        'ns': {'prior': {'min': 0.9, 'max': 1.1}, 'latex': 'n_s'},
        'noise_std_pixel': {
            'prior': {'dist': 'norm', 'loc': 20, 'scale': 5},
            'latex': r'\sigma_\mathrm{pix}'},
        # Derived
        'Map_Cl_at_500': {'latex': r'C_{500,\,\mathrm{map}}'}},
    'likelihood': {'my_cl_like': {
        "external": my_like,
        # Declare required quantities!
        "requires": {'Cl': {'tt': l_max}},
        # Declare derived parameters!
        "output_params": ['Map_Cl_at_500']}},
    'theory': {'camb': {'stop_at_error': True}},
    'packages_path': packages_path},
    # sampler
    "sampler": {
        "mcmc": {"Rminus1_stop": 0.001, "max_tries": 10000}}}


IndentationError: unexpected indent (2614391243.py, line 23)

In [28]:
from cobaya import run
updated_info, sampler = run (info)

[run] *ERROR* You need to specify a sampler using the 'sampler' key as e.g. `sampler: {mcmc: None}.`


LoggedError: You need to specify a sampler using the 'sampler' key as e.g. `sampler: {mcmc: None}.`